# 4일차 – 결정 트리 및 트리 앙상블 심화 강의

---

## 1. 결정 트리(Decision Tree) 개요

- **개념**  
  - 데이터를 분할(split)하며 학습하는 트리 구조 모델  
  - 노드에서 **특성 하나**를 기준으로 분기 → 리프 노드에서 예측  

- **핵심 하이퍼파라미터**  
  1. **criterion**: 분할 평가 지표  
     - `gini` (Gini 불순도)  
     - `entropy` (정보 이득)  
  2. **max_depth**: 트리 최대 깊이 (복잡도 제어)  
  3. **min_samples_split**: 자식 노드 분할 최소 샘플 수  
  4. **min_samples_leaf**: 리프 노드가 가져야 할 최소 샘플 수  

- **장단점**  
  - 장점: 직관적, 전처리(스케일링) 불필요, 범주형과 연속형 모두 처리  
  - 단점: 과적합 경향, 작은 데이터 변화에도 구조 민감  

---

## 2. 앙상블(Ensemble) 보강 주제

1. **Bagging (Bootstrap Aggregating)**  
   - 여러 결정 트리를 **독립 학습** → 예측 다수결(분류) 또는 평균(회귀)  
   - 대표: `RandomForestClassifier`

2. **Boosting**  
   - 약한 학습기(Decision Stump)를 순차 학습하며 오류 보완  
   - 대표: `AdaBoostClassifier`, `GradientBoostingClassifier`, `XGBClassifier`

3. **랜덤 포레스트 핵심 하이퍼파라미터**  
   - `n_estimators`: 트리 개수  
   - `max_features`: 분할할 특성 수 비율(√p, log2 p…)  
   - `bootstrap`: 부트스트랩 샘플링 여부

4. **부스팅 핵심 하이퍼파라미터**  
   - `learning_rate`: 각 트리가 기여하는 정도  
   - `n_estimators`: 부스팅 단계 수  
   - `max_depth` 등 트리 복잡도 제어  

5. **특성 중요도(Feature Importance)**  
   - 트리 기반 모델이 자동 계산  
   - 시각화하여 중요한 특성 확인  

---




# 앙상블 트리 모델 비교: Random Forest, Gradient Boosting, Hist-GB, LightGBM, XGBoost

---

## 1. Random Forest (RF)

### 개념  
- **Bagging**: 여러 개의 결정 트리를 서로 다른 부트스트랩 샘플과 무작위 특성 서브셋으로 학습  
- **예측**: 분류는 투표, 회귀는 평균

### 장점  
- 과적합 억제: 서로 다른 트리 투표로 노이즈 완화  
- 적은 전처리: 특성 스케일링 불필요, 결측치 일부 자동 처리  
- 직관적 해석: 특성 중요도 제공  

### 단점  
- 많은 트리→메모리·시간 비용 증가  
- 복잡한 모델 → 예측 지연(latency)  
- 트리 수가 너무 많으면 diminishing returns

### 사용 용도  
- **중소규모** tabular 데이터 전반에 강건  
- **특성 수 많음**(수십~수백) & 일부 노이즈 있는 경우  
- 빠른 프로토타이핑·베이스라인에서 우선 적용  

---

## 2. Gradient Boosting Machine (GBM)

### 개념  
- **Boosting**: 약한 학습기(깊이 작 은 트리)를 순차 학습  
- **각 단계**: 이전 모델 오차(잔차)에 초점을 맞춰 새 트리 학습 → 누적 예측 합산

### 장점  
- 높은 예측 성능: 복잡한 비선형 관계 캡처  
- 유연성: 손실 함수(custom loss) 지정 가능  
- 과적합 제어: 학습률(learning_rate), 트리 깊이 등 조절

### 단점  
- 느린 학습: 순차적 학습 구조  
- 하이퍼파라미터 많음: 튜닝 복잡  
- 민감한 과소적합/과대적합 트레이드오프

### 사용 용도  
- **고정밀 예측**이 필요한 금융 리스크, 마케팅 응답 예측  
- **중간 규모**(수만 행 이하) 데이터에서 최적화 성능 요구 시  

---

## 3. Hist-Gradient Boosting (Hist-GB)

### 개념  
- 입력 연속값을 **히스토그램 빈**으로 양자화 → 트리 분할 시 상수 시간 계산  
- LightGBM의 아이디어와 유사

### 장점  
- **속도**: 분할 후보 수 축소 → 대규모 데이터에서도 빠름  
- **메모리 효율**: 히스토그램 구조로 압축 저장  
- 결측치 자동 처리, 카테고리 엔코딩 내장

### 단점  
- 양자화 손실: 매우 경계적 분할 필요 시 정보 손실 가능  
- 하이퍼파라미터(빈 개수) 추가

### 사용 용도  
- **수백만 행** 이상의 대규모 tabular 데이터  
- 빠른 반복 실험 & 실시간 예측 파이프라인  

---

## 4. LightGBM

### 개념  
- Microsoft 개발, **Leaf-wise 성장** 전략(가장 큰 손실 감소 분할 우선)  
- **Histogram 기반** 분할, GPU 지원

### 장점  
- **초고속 학습**: 대규모 데이터 & 특성 많아도 스케일  
- **높은 정확도**: Leaf-wise 분할이 더 세밀한 모델링  
- 낮은 메모리 사용, 병렬 처리 최적화

### 단점  
- 작은 데이터/특성에서는 과적합 위험↑  
- Leaf-wise 분할 → 불균형 트리 생성 시 일반화 저하  
- 파라미터 튜닝 복잡(leaf 헬스 조정 필요)

### 사용 용도  
- **수백만~수천만 샘플**, **수백 특성**  
- 실시간 온라인 서빙 시스템 (GPU, 멀티스레드 활용)  
- 경쟁용 캐글/대회 상위권 모델

---

## 5. XGBoost

### 개념  
- “eXtreme” GBM 구현 – **Second-order Taylor expansion** 기반 손실 근사  
- 정교한 **regularization**(L1/L2), **tree pruning**

### 장점  
- 매우 높은 안정성 & 정확도  
- **과적합 제어**: gamma, lambda, alpha 조절  
- 분산 학습(spark), GPU 지원

### 단점  
- 구현 복잡도: 다양한 파라미터  
- 학습 속도: 기본은 LightGBM보다 느림  
- 메모리 사용 높음  

### 사용 용도  
- **고품질 예측** 필수 금융·의료·광고 클릭률 예측  
- 병렬·분산 환경에서 대규모 학습  
- 커스텀 목표/평가 지표 필요 시  

---

## 6. 요약 비교 테이블

| 모델                | 규모        | 속도       | 정확도     | 튜닝 난이도 | 특징                         |
|-------------------|-----------|----------|---------|----------|----------------------------|
| Random Forest     | 중소       | 빠름       | 보통~높음  | 보통       | 전처리 적음, 베이스라인용        |
| Gradient Boosting | 중소~중   | 느림       | 높음      | 높음       | 손실 함수 유연, 순차 학습         |
| Hist-GB           | 대규모     | 매우 빠름   | 높음      | 중간       | 히스토그램 양자화, 빠른 실험      |
| LightGBM          | 대규모     | 초고속     | 매우 높음  | 높음       | leaf-wise, GPU, 캐글 상위권 노하우 |
| XGBoost           | 중대규모   | 빠름       | 매우 높음  | 높음       | 2차 근사, 강력한 regularization  |

---

> **실전 팁**  
> 1. **베이스라인**: Random Forest → 빠르게 성능 확인  
> 2. **정밀 튜닝**: XGBoost / LightGBM → 최종 모델  
> 3. **대규모**: Hist-GB / LightGBM → 효율 반복  
> 4. **작은 샘플**: Gradient Boosting → 손실 함수 맞춤  
> 5. **특성 선택**: Random Forest의 feature_importances_ 활용  


# 앙상블 트리 모델별 주요 하이퍼파라미터

---

## 1. Random Forest  
- **n_estimators**: 생성할 결정 트리 수  
- **max_depth**: 각 트리의 최대 깊이  
- **max_features**: 분할 시 고려할 특성 수 (`"sqrt"`, `"log2"`, 비율 또는 정수)  
- **min_samples_split**: 내부 노드 분할 최소 샘플 수  
- **min_samples_leaf**: 리프 노드 최소 샘플 수  
- **bootstrap**: 부트스트랩 샘플링 사용 여부 (`True`/`False`)  
- **class_weight** (분류 시): 불균형 클래스 가중치

---

## 2. Gradient Boosting (sklearn.GradientBoosting)  
- **n_estimators**: 부스팅 단계(트리) 수  
- **learning_rate**: 각 트리 기여도 축소 계수  
- **max_depth**: 개별 트리 최대 깊이  
- **subsample**: 각 트리 학습에 사용할 샘플 비율 (0<subsample≤1)  
- **max_features**: 각 분할에 고려할 특성 수  
- **min_samples_split** / **min_samples_leaf**: 트리 복잡도 제어  
- **loss**: 손실 함수 (`'deviance'`(logistic), `'exponential'`(AdaBoost 유사) 등)

---

## 3. HistGradientBoosting (sklearn.HistGradientBoosting)  
- **max_iter**: 트리(부스팅 단계) 수  
- **learning_rate**: 각 단계 학습률  
- **max_leaf_nodes**: 트리 당 최대 리프 수 (대신 max_depth 사용 가능)  
- **max_depth**: 트리 최대 깊이 (선택적)  
- **min_samples_leaf**: 리프 최소 샘플 수  
- **l2_regularization**: L₂ 패널티 강도  
- **max_bins**: 연속 특성 히스토그램 빈 개수  
- **categorical_features**: 범주형 피처 인덱스

---

## 4. LightGBM (`lightgbm.LGBMClassifier/Regressor`)  
- **n_estimators**: 트리 수  
- **learning_rate**: 학습률  
- **num_leaves**: 트리의 최대 리프 수 (leaf-wise 분할에 핵심)  
- **max_depth**: 트리 최대 깊이  
- **min_data_in_leaf**: 리프당 최소 데이터 수  
- **feature_fraction**: 매 트리마다 사용할 특성 비율  
- **bagging_fraction** & **bagging_freq**: 데이터 샘플링 비율 및 빈도  
- **lambda_l1** / **lambda_l2**: L₁ / L₂ 정규화 항 강도  
- **min_gain_to_split**: 분할을 허용할 최소 정보 이득

---

## 5. XGBoost (`xgboost.XGBClassifier/Regressor`)  
- **n_estimators**: 트리 수  
- **learning_rate (eta)**: 학습률  
- **max_depth**: 트리 최대 깊이  
- **subsample**: 각 트리 샘플링 비율  
- **colsample_bytree**, **colsample_bylevel**: 특성 샘플링 비율  
- **gamma**: 최소 분할 손실 감소량 (pruning 강도)  
- **reg_alpha** / **reg_lambda**: L₁ / L₂ 정규화 강도  
- **min_child_weight**: 자식 노드 최소 가중치 합  
- **tree_method**: 트리 구축 방식 (`'auto'`, `'hist'`, `'gpu_hist'` 등)

---

> **팁**:  
> - 처음에는 **n_estimators**와 **learning_rate**를 고정(예: 100/0.1)하고,  
>   `max_depth`와 샘플링 파라미터를 탐색한 뒤,  
> - `subsample`/`colsample_*`을 조정 → 마지막으로 정규화(λ계수) 세팅  
> - **GridSearchCV**나 **RandomizedSearchCV**로 단계별 튜닝을 권장합니다.  


# 실습 과제

1. 단일 트리 vs 랜덤 포레스트 비교

* max_depth를 1,3,5,None 으로 바꿔가며 결정 트리 성능 관찰

* 최적 RandomForestClassifier 하이퍼파라미터를 GridSearchCV로 탐색

2. AdaBoost 실험

* AdaBoostClassifier(n_estimators=[50,100], learning_rate=[0.01,0.1,1])

* 테스트 정확도 비교

3. Gradient Boosting vs XGBoost

* GradientBoostingClassifier와 XGBClassifier 성능·속도 비교

4. 특성 중요도 활용

* 상위 5개 특징만 선택하여 모델 재학습 → 성능 변화 확인

5. 앙상블 시각화

* 단일 트리와 포레스트의 결정 경계를 2D 데이터(make_blobs)로 비교 시각화

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV

# 1) DT max_depth 비교
for d in [1,3,5,None]:
    dt = DecisionTreeClassifier(max_depth=d, random_state=0)
    dt.fit(X_tr, y_tr)
    print(f"DT depth={d}, Acc={accuracy_score(y_te, dt.predict(X_te)):.3f}")

# 2) AdaBoost GridSearch
param_ab = {'n_estimators':[50,100], 'learning_rate':[0.01,0.1,1]}
gs_ab = GridSearchCV(AdaBoostClassifier(random_state=0), param_ab, cv=5)
gs_ab.fit(X_tr, y_tr)
print("AB Best:", gs_ab.best_params_, "Acc:", accuracy_score(y_te, gs_ab.predict(X_te)))

# 3) XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
xgb.fit(X_tr, y_tr)
print("XGB Acc:", accuracy_score(y_te, xgb.predict(X_te)))

# 4) 상위 5개 특성만 재학습
top5 = load_breast_cancer().feature_names[idx[:5]]
rf5 = RandomForestClassifier(**gs_rf.best_params_, random_state=0)
rf5.fit(X_tr[:, idx[:5]], y_tr)
print("RF top5 Acc:", accuracy_score(y_te, rf5.predict(X_te[:, idx[:5]])))

# 5) 2D 경계 시각화
X2, y2 = make_blobs(n_samples=200, centers=2, random_state=0)
fig, axes = plt.subplots(1,2, figsize=(10,4))
for ax, model, title in zip(axes,
                            [DecisionTreeClassifier(max_depth=3).fit(X2, y2),
                             RandomForestClassifier(n_estimators=50).fit(X2, y2)],
                            ['Tree','Forest']):
    xx, yy = np.meshgrid(np.linspace(X2[:,0].min()-1, X2[:,0].max()+1,200),
                         np.linspace(X2[:,1].min()-1, X2[:,1].max()+1,200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3)
    ax.scatter(X2[:,0], X2[:,1], c=y2, edgecolor='k')
    ax.set_title(title)
plt.show()


# 4일차 실습 과제 (와인 데이터 기반)

**데이터**: `wine.csv`  
- 특성: 화학 성분(feature columns)  
- 타깃: `quality` 또는 `type` (데이터 컬럼 구성에 따라 조정)

---

## 1. 결정 트리 실습
- `DecisionTreeClassifier` / `DecisionTreeRegressor` 적용  
- `max_depth` = [2, 4, 6, None] 변경 → 테스트 성능(정확도 or R²) 확인  

## 2. 랜덤 포레스트 튜닝
- `RandomForestClassifier` / `RandomForestRegressor`  
- GridSearchCV:  
  - `n_estimators`: [50, 100, 200]  
  - `max_features`: ['sqrt', 0.3, 0.5]  
  - `max_depth`: [None, 10, 20]  
- 최적 파라미터로 테스트 세트 평가  

## 3. AdaBoost 실험
- `AdaBoostClassifier` / `AdaBoostRegressor`  
- `n_estimators` = [50, 100]  
- `learning_rate` = [0.01, 0.1, 1]  
- 테스트 세트 성능 비교  

## 4. 특성 중요도 분석
- 최적 모델의 `feature_importances_` 추출  
- 상위 8개 특성 시각화  

## 5. 앙상블 경계 시각화 (Optional)
- 2개 특성(예: alcohol vs pH) 선택  
- 결정 트리 vs 랜덤 포레스트의 2D 결정 경계 비교  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# 0) 데이터 로드 & 특성/타깃 분리
df = pd.read_csv('/mnt/data/wine.csv')
# 예시: 'quality'를 회귀 타깃으로, 클래식 분류는 'type'으로 설정
X = df.drop(columns=['quality'])
y_reg = df['quality']
# 회귀가 아닌 분류 과제로 'type' 컬럼이 있을 때:
# y_clf = df['type']

# 1) train/test 분할 & 스케일링
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y_reg, test_size=0.3, random_state=42
)
scaler = StandardScaler().fit(X_tr)
X_tr_s, X_te_s = scaler.transform(X_tr), scaler.transform(X_te)

# 2) 결정 트리 실습 (회귀 예시)
print("Decision Tree Regression:")
for depth in [2, 4, 6, None]:
    dt = DecisionTreeClassifier(max_depth=depth, random_state=0)
    dt.fit(X_tr_s, y_tr.astype(int))  # 분류 시
    acc = dt.score(X_te_s, y_te.astype(int))
    print(f" max_depth={depth}: Test Accuracy = {acc:.3f}")

# 3) 랜덤 포레스트 튜닝
rf = RandomForestClassifier(random_state=0)
param_rf = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 0.3, 0.5],
    'max_depth': [None, 10, 20]
}
gs_rf = GridSearchCV(rf, param_rf, cv=5, scoring='accuracy', n_jobs=-1)
gs_rf.fit(X_tr_s, y_tr.astype(int))
best_rf = gs_rf.best_estimator_
print("\nRandom Forest Best Params:", gs_rf.best_params_)
print("RF Test Accuracy:", f"{best_rf.score(X_te_s, y_te.astype(int)):.3f}")

# 4) AdaBoost 실험
print("\nAdaBoost Classification:")
for n in [50, 100]:
    for lr in [0.01, 0.1, 1]:
        ab = AdaBoostClassifier(n_estimators=n, learning_rate=lr, random_state=0)
        ab.fit(X_tr_s, y_tr.astype(int))
        print(f" n={n}, lr={lr}: Test Accuracy = {ab.score(X_te_s, y_te.astype(int)):.3f}")

# 5) 특성 중요도 분석 (Random Forest 예시)
importances = best_rf.feature_importances_
feat_names = X.columns
idx = np.argsort(importances)[::-1][:8]
plt.figure(figsize=(8,5))
plt.barh(feat_names[idx], importances[idx])
plt.gca().invert_yaxis()
plt.xlabel("Importance"); plt.title("Top 8 Feature Importances")
plt.show()

# 6) Optional: 2D 경계 시각화
# select two features, e.g., first two:
X2 = X_tr_s[:, idx[:2]]
y2 = y_tr.astype(int)
xx, yy = np.meshgrid(
    np.linspace(X2[:,0].min()-1, X2[:,0].max()+1, 200),
    np.linspace(X2[:,1].min()-1, X2[:,1].max()+1, 200)
)
fig, axes = plt.subplots(1, 2, figsize=(10,4))
for ax, model, name in zip(
    axes, [DecisionTreeClassifier(max_depth=4), best_rf],
    ['Decision Tree','Random Forest']
):
    model.fit(X2, y2)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3)
    ax.scatter(X2[:,0], X2[:,1], c=y2, edgecolor='k')
    ax.set_title(name)
plt.show()
